In [ ]:
# -------- REPORT TEMPLATE -------- #
# This template provides statworx CI palettes and helpful functions for formatting graphs and tables.
# Run the imports and follow the sequentially numbered steps.

# Use the following commands in the terminal to generate the html output:
# pip install [--upgrade] nbconvert (version 6.x.x needed)
# jupyter nbconvert --output-dir="." --output "output.html" --execute --no-input report_template.ipynb --to html

%load_ext autoreload
%autoreload 2

In [ ]:
# IMPORTS --------------------------------------------------------
import warnings

warnings.simplefilter("ignore")
import base64
from datetime import date
from xml.dom import minidom

import matplotlib as mpl
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import HTML, SVG, Markdown, display
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

# center stuff
css = """
.output {
    /* align-items: center; */
    color: #283440;
    font-family: Arial;
    font-size: 11pt;
    text-align: justify;
}

/* Table */
table.dataframe thead {
    color: #0000FF;
    border-bottom: 1px solid #0000FF;
}
table.dataframe tbody{
    color: #283440;
}

/* entry hover */
table.dataframe tbody td:hover {
  color: #0000FF;
  background-color: #F2F3FF;
}

/* row hover */
table.dataframe tbody tr:hover {
    background-color: #F2F3FF;
}

/* Header */
.h1 {
    border-top: 3px solid #0000FF;
    color: #0000FF;
    font-size: 24px;
    font-weight: bold;
    height: 35px;
    line-height: 35px;
    text-align: center;
}
.h2 {
    border: 2px solid black;
    border-bottom-color: #00000000;
    border-right-color: #00000000;
    border-left-color: #00000000;
    color: black;
    font-size: 18px;
    font-weight: bold;
    height: 40px;
    line-height: 30px;
    text-align: center;
    text-shadow: 1px 1px #ffffff;
}
.h3 {
    color: black;
    font-size: 16px;
    font-weight: bold;
    line-height: 30px;
}

/* Body */
.body {
    color: #283440;
    font-family: Arial;
    font-size: 11pt;
    text-align: justify;
}
element.style {
    color: #283440;
    font-family: Arial;
}

/* Widgets */
.jupyter-widgets.widget-tab > .widget-tab-contents {
    border-top-color: #0000FF;
    border-left-color: #00000000;
    border-right-color: #00000000;
    border-bottom-color: #00000000;
}
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    background: white;
    border: None;
    color: #283440;
    text-align: center;
} 
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab.p-mod-current {
    background: white;
    border: 1px solid #0000FF;
    border-bottom-color: #00000000;
    color: #0000FF;
    text-align: center;
}

"""

HTML(f"<style>{css}</style>")

In [ ]:
# 1. Run the following cells in order to load needed functions and statworx CI color palettes. There is no
# need to make any changes here.

# Functions to load and manipulate landing page


def load_landing_page(path_landing_page="landing_page_blueKV_logo.svg"):
    """Loads a template.

    Args:
        path_landing_page (str): Template path. Use one of the following:
                            - landing_page_blackKV_logo.svg
                            - landing_page_blackKV_slogan_logo.svg
                            - landing_page_blackKV_slogan.svg
                            - landing_page_blackKV.svg
                            - landing_page_blueKV_logo.svg
                            - landing_page_blueKV_slogan_logo.svg
                            - landing_page_blueKV_slogan.svg
                            - landing_page_blueKV.svg

                            blackKV/blueKV: Key visual is black/blue.
                            slogan: "We create the next" slogan is displayed.
                            logo: Template contains free space for client logo.

    Returns:
        str: SVG code.
    """
    with open(path_landing_page) as file:
        svg = file.read().replace("\n", "")
    return svg


def fill_template(
    svg,
    title1="",
    title2="",
    line1="",
    line2="",
    line3="",
    set_date_to_today=True,
    english_date=True,
    other_date="",
):
    """Fills template. Note that if an empty string is specified, line is not shown in template.

    Args:
        svg (str): SVG code.
        title1 (str): Title.
        title2 (str): Subtitle.
        line1 (str): First line.
        line2 (str): Second line.
        line3 (str): Third line.
        set_date_to_today (bool): If true, today's date is inserted in template.
        english_date (bool): If true, english date format is used. If false, german date format is used.
        other_date (str): If set_date_to_today is set to False, this date is inserted.

    Returns:
        str: SVG code.
    """
    if set_date_to_today and english_date:
        svg_date = f"Date: {date.today()}"
    elif set_date_to_today and not english_date:
        svg_date = f"Datum: {date.today().strftime('%d.%m.%Y')}"
    else:
        svg_date = other_date

    svg = (
        svg.replace("TITLE1", title1)
        .replace("TITLE2", title2)
        .replace("LINE1", line1)
        .replace("LINE2", line2)
        .replace("LINE3", line3)
        .replace("DATE", svg_date)
    )
    return svg


def insert_client_logo(svg, logo_path, scale=1, pos_x=0, pos_y=0):
    """Inserts client logo in upper left corner of SVG.

    Args:
        svg (str): SVG code.
        logo_path (str): Path to client logo (jpg or png is supported).
        scale (float): Change size of logo.
        pos_x (float): Move client logo along the x-axis. If positive, logo is moved upwards.
        pos_y (float): Move client logo along the y-axis. If positive, logo is moved to the right.

    Returns:
        str: SVG code.
    """
    doc = minidom.parseString(svg)

    logo_encoded = base64.b64encode(open(logo_path, "rb").read())
    logo_encoded = str(logo_encoded).replace("b'", "").replace("'", "")

    image_tag = doc.createElement("image")
    image_tag.setAttribute(
        "xlink:href", f"data:image/{logo_path.split('.')[-1]}; base64, {logo_encoded}"
    )
    image_tag.setAttribute("height", str(200 * scale))
    image_tag.setAttribute("width", str(200 * scale))
    image_tag.setAttribute("transform", f"translate({10+pos_x}, {-20-pos_y})")

    elements = doc.getElementsByTagName("svg")
    elements[-1].appendChild(image_tag)

    return doc.toxml()

In [ ]:
# Functions to construct LinearSegmentedColormap


def get_continuous_cmap(hex_list, float_list=None, with_transparency=False):
    """Creates and returns a color map that can be used in heat map figures.
    - If float_list is not provided, colour map graduates linearly between each color in hex_list.
    - If float_list is provided, each color in hex_list is mapped to the respective location in float_list.

    Args:
        hex_list (list[str]): Hex code.
        float_list (list[float]): Floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        with_transparency(bool): If true, color gets transparent in the middle of the range.

    Returns:
        LinearSegmentedColormap: Color map.
    """
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0, 1, len(rgb_list)))

    cdict = dict()
    for num, col in enumerate(["red", "green", "blue"]):
        col_list = [
            [float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))
        ]
        cdict[col] = col_list

    if with_transparency:
        cdict = {**cdict, "alpha": ((0, 1.0, 1.0), (0.5, 0.5, 0.5), (1.0, 1.0, 1.0))}

    cmp = mcolors.LinearSegmentedColormap("my_cmp", segmentdata=cdict, N=256)

    return cmp


def hex_to_rgb(value):
    """Converts hex to rgb colors.

    Args:
        value (str): 6 characters representing a hex colour.

    Returns:
        list: List of RGB values with length 3.
    """
    value = value.strip("#")  # removes hash symbol if present
    lv = len(value)
    return tuple(int(value[i : i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_dec(value):
    """Converts rgb to decimal colours (i.e. divides each value by 256).

    Args:
        value (list): RGB values with length 3.

    Returns:
        list: Decimal values with length 3.
    """
    return [v / 256 for v in value]

In [ ]:
# Function to change default colormap


def change_default_colormap(palette):
    """Set default colormap of notebook.

    Args:
        palette (cmap): Colormap.

    """
    # construct and register ListedColormap
    cmap = mcolors.ListedColormap(palette)
    mpl.colormaps.register(cmap=cmap, name="statworx_cmap")

    # change default colormap
    mpl.rc("image", cmap="statworx_cmap")
    sns.set_palette(palette)

In [ ]:
# construct ListedColormap

colors = [
    "#0000FF",
    "#000000",
    "#FFFFFF",
    "#283440",
    "#6C7D8C",
    "#B6BDCC",
    "#EBF0F2",
    "#FE0D6C",
    "#00C800",
    "#FFFF00",
]
statworx_palette = sns.color_palette(colors, as_cmap=True)

colors_without_white = [
    "#0000FF",
    "#000000",
    "#283440",
    "#6C7D8C",
    "#B6BDCC",
    "#EBF0F2",
    "#FE0D6C",
    "#00C800",
    "#FFFF00",
]
statworx_palette_without_white = sns.color_palette(colors_without_white, as_cmap=True)

blue_black_pink = ["#0000FF", "#000000", "#FE0D6C"]
statworx_palette_blue_black_pink = sns.color_palette(blue_black_pink, as_cmap=True)

blue_black_green = ["#0000FF", "#000000", "#00C800"]
statworx_palette_blue_black_green = sns.color_palette(blue_black_green, as_cmap=True)

colors_good_to_bad = ["#0000FF", "#000000", "#283440", "#6C7D8C", "#FE0D6C"]
statworx_palette_good_to_bad = sns.color_palette(colors_good_to_bad, as_cmap=True)

colors_blue = ["#0000FF", "#3343FF", "#6573FF", "#98A3FF", "#CCD1FF", "#F2F3FF"]
statworx_palette_blue = sns.color_palette(colors_blue, as_cmap=True)

colors_black = ["#000000", "#333333", "#666666", "#999999", "#CCCCCC", "#F2F2F2"]
statworx_palette_black = sns.color_palette(colors_black, as_cmap=True)

# construct LinearSegmentedColormap

colors_blue_black = ["#0000FF", "#000000"]
statworx_palette_blue_black = sns.color_palette(colors_blue_black, as_cmap=True)

colors_blue_white = ["#0000FF", "#FFFFFF"]
statworx_palette_blue_white = sns.color_palette(colors_blue_white, as_cmap=True)

cmap_blue_black = get_continuous_cmap(statworx_palette_blue_black, with_transparency=True)
cmap_blue_white = get_continuous_cmap(statworx_palette_blue_white)

In [ ]:
print("statworx_palette:")
sns.color_palette(colors)

In [ ]:
print("statworx_palette_without_white:")
sns.color_palette(colors_without_white)

In [ ]:
print("statworx_palette_blue_black_pink:")
sns.color_palette(blue_black_pink)

In [ ]:
print("statworx_palette_blue_black_green:")
sns.color_palette(blue_black_green)

In [ ]:
print("statworx_palette_good_to_bad:")
sns.color_palette(colors_good_to_bad)

In [ ]:
print("statworx_palette_blue:")
sns.color_palette(colors_blue)

In [ ]:
print("statworx_palette_black:")
sns.color_palette(colors_black)

In [ ]:
print("cmap_blue_black")
cmap_blue_black

In [ ]:
print("cmap_blue_white")
cmap_blue_white

In [ ]:
# Style attributes and functions for tables

headers_black = {
    "selector": "th:not(.index_name)",
    "props": "background-color: #000000; color: white;",
}
headers_gray = {
    "selector": "th:not(.index_name)",
    "props": "background-color: #B6BDCC; color: black;",
}
headers_blue = {
    "selector": "th:not(.index_name)",
    "props": "background-color: #0000ff; color: white;",
}
headers_black_font = {"selector": "th:not(.index_name)", "props": "color: #000000;"}
headers_blue_font = {"selector": "th:not(.index_name)", "props": "color: #0000ff;"}
cell_hover = {"selector": "td:hover", "props": "color: #0000ff;"}
row_hover = {  # highlights full row
    "selector": "tr:hover",
    "props": "background-color: #F2F3FF;",
}
transparent_row_hover = {  # overwrites default row hover
    "selector": "tr:hover",
    "props": "background-color: transparent;",
}
index_names_black = {
    "selector": ".index_name",
    "props": "font-style: italic; color: #000000; font-weight:normal;",
}
index_names_gray = {
    "selector": ".index_name",
    "props": "font-style: italic; color: #6C7D8C; font-weight:normal;",
}


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


def highlight_min(s, props=""):
    return np.where(s == np.nanmin(s.values), props, "")


def highlight_neg_values(s, props=""):
    return np.where(s < 0, props, "")


def highlight_large_values(s, value=100, props=""):
    return np.where(s >= value, props, "")

In [ ]:
# 2. Load your favourite landing page template. See docstring of function load_landing_page() for further information.

svg = load_landing_page(path_landing_page="../landing_page/landing_page_blueKV_logo.svg")

# 3. Fill template with the desired text. See docstring of fill_template() for a description of each parameter.

svg = fill_template(
    svg,
    title1="Title",
    title2="Subtitle",
    line1="line1",
    line2="line2",
    line3="line3",
    set_date_to_today=True,
    english_date=True,
)

# 4. [OPTIONAL] If you want to add the client logo in the upper left corner, download the client logo in png or jpg
# format and load it here. The parameters scale, pos_x and pos_y can be modified if you would like to scale or move
# the logo.

svg = insert_client_logo(
    svg, logo_path="../logos/statworx-Logo-Black.png", scale=0.4, pos_x=10, pos_y=-30
)

# 5. Display landing page.

SVG(svg)

In [ ]:
# 6. Insert your preferred colormap from above. It will be the default colormap in this notebook.

change_default_colormap(palette=statworx_palette_blue_black_pink)

# 7. Add table of contents (if you like) and header between sections

# <div class="h1">Table of Contents</div>

* [Chapter 1](#bullet_1)
    * [Chapter 1.1](#bullet_1.1)
        * [Chapter 1.1.1](#bullet_1.1.1)
        * [Chapter 1.1.2](#bullet_1.1.2)
    * [Chapter 1.2](#bullet_1.2)
* [Plot Formatting](#bullet_2)
* [Table Formatting](#bullet_3)
    * [Default (CSS)](#bullet_3.1)
    * [Pandas Style Method](#bullet_3.2)

# <div class="h1 anchor" id="bullet_1">Chapter 1</div>

<div class="body">Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren.</div>

## <div class="h2 anchor" id="bullet_1.1">Chapter 1.1 </div>

<div class="body">Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren.</div>

### <div class="h3 anchor" id="bullet_1.1.1">Chapter 1.1.1 </div>

<div class="body">Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren.</div>

### <div class="h3 anchor" id="bullet_1.1.2">Chapter 1.1.2 </div>

<div class="body">Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren.</div>

## <div class="h2 anchor" id="bullet_1.2">Chapter 1.2 </div>

<div class="body">Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren.</div>

# <div class="h1 anchor" id="bullet_2">Plots</div>

<div class="body">Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren. Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren.</div>

In [ ]:
# In the following, you can find some example plots with statworx colors.

# !!!
# Note that according to the statworx brand manual, it is not allowed to use two of
# the additional colors (pink, green, yellow) in the same plot.
# !!!

# If you get a SSL error when loading the seaborn datasets, try the following:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

df = sns.load_dataset("penguins")
sns.pairplot(df, hue="species")
plt.show()

In [ ]:
rs = np.random.RandomState(11)
x = rs.gamma(2, size=1000)
y = -0.5 * x + rs.normal(size=1000)
sns.jointplot(x=x, y=y, kind="hex")
plt.show()

In [ ]:
%matplotlib inline
import ipywidgets as widgets

diamonds = sns.load_dataset("diamonds")

out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab(children=[out1, out2, out3])
tab.set_title(0, "good-to-bad palette")
tab.set_title(1, "blue palette")
tab.set_title(2, "black palette")
display(tab)

with out1:
    f, ax = plt.subplots(figsize=(7, 5))
    sns.histplot(
        diamonds,
        x="price",
        hue="cut",
        multiple="stack",
        palette=statworx_palette_good_to_bad[:5],
        edgecolor=".3",
        linewidth=0.5,
        log_scale=True,
    )
    ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
    plt.show()

with out2:
    f, ax = plt.subplots(figsize=(7, 5))
    sns.histplot(
        diamonds,
        x="price",
        hue="cut",
        multiple="stack",
        palette=statworx_palette_blue[:5],
        edgecolor=".3",
        linewidth=0.5,
        log_scale=True,
    )
    ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
    plt.show()

with out3:
    f, ax = plt.subplots(figsize=(7, 5))
    sns.histplot(
        diamonds,
        x="price",
        hue="cut",
        multiple="stack",
        palette=statworx_palette_black[:5],
        edgecolor=".3",
        linewidth=0.5,
        log_scale=True,
    )
    ax.xaxis.set_major_formatter(mpl.ticker.ScalarFormatter())
    plt.show()

In [ ]:
flights_long = sns.load_dataset("flights")
flights = flights_long.pivot("month", "year", "passengers")
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(flights, annot=True, fmt="d", linewidths=0.5, ax=ax, cmap=cmap_blue_black)
plt.show()

## <div class="h2 anchor" id="bullet_3.1">Default (CSS)</div>

In [ ]:
# The CSS code at the beginning of this notebook specifies how tables are formatted.
# This is how it looks like:
pd.DataFrame(np.random.randn(4, 4), columns=["a", "b", "c", "d"])

## <div class="h2 anchor" id="bullet_3.2">Pandas Style Method</div>

In [ ]:
# If you want to have more fancy tables, you can use pandas style method. In the following,
# you can find some examples. The style attributes were defined at the beginning of this
# notebook. Note: If you get an error message, upgrade pandas to the newest version (or 1.4.0).

df1 = pd.DataFrame(
    [[38.0, 2.0, 18.0, 22.0], [19, 439, 6, 452]],
    index=pd.Index(["row_0", "row_1"], name="Actual Label:"),
    columns=pd.MultiIndex.from_product(
        [["col_0", "col_1"], ["a", "b"]], names=["Model:", "Predicted:"]
    ),
)

df2 = pd.DataFrame(np.random.randn(4, 4), columns=["a", "b", "c", "d"])

<br>Element hover:

In [ ]:
df1.style.format(precision=2).set_table_styles(
    [headers_blue, cell_hover, row_hover, index_names_gray]
)

In [ ]:
df1.style.format(precision=2).set_table_styles(
    [headers_black, cell_hover, row_hover, index_names_gray]
)

In [ ]:
df1.style.format(precision=2).set_table_styles(
    [headers_gray, cell_hover, row_hover, index_names_gray]
)

In [ ]:
df1.style.format(precision=2).set_table_styles(
    [headers_blue_font, cell_hover, row_hover, index_names_gray]
)

In [ ]:
df1.style.format(precision=2).set_table_styles([cell_hover, row_hover, index_names_gray])

<br>Highlight values larger than 100:

In [ ]:
df1.style.format(precision=2).set_table_styles([row_hover, index_names_black]).apply(
    highlight_large_values, props="color:#FFFFFF;background-color:#0000ff", axis=0
)

<br>Highlight negative values:

In [ ]:
df2.style.format(precision=2).set_table_styles([row_hover]).apply(
    highlight_neg_values, props="color:#FFFFFF;background-color:#0000ff", axis=0
)

<br> Highlight depending on value:

In [ ]:
df2.style.set_table_styles([row_hover]).background_gradient(cmap=cmap_blue_white)